### Monkey Patching меняет поведение функции динамически

In [17]:
class MyClass():
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def add(self):
        return self.a + self.b

my = MyClass(3, 4)
print(my.add())
MyClass.speak = "OK"  #Monkey Patching
print(my.speak)

7
OK


### Можем сделать monkey patching с lambda

In [15]:
MyClass.speak2 = lambda self, message: "This is a message: {}".format(message)
print(my.speak2("My Message"))

This is a message: My Message


<b>На примере класса Fraction модуля fractions</b>

In [26]:
from fractions import Fraction

f = Fraction(2, 3)
print(f)
print(f.denominator)
print(f.numerator)

2/3
3
2


In [16]:
Fraction.speak = 100 #Monkey Patching
print(f.speak)

100


<b>Более полезное применение</b>

In [33]:
Fraction.is_integral = lambda self: self.denominator == 1
f1 = Fraction(2, 3)
f2 = Fraction(64, 8)
print(f1.is_integral())
print(f2.is_integral())

False
True


<b>Декорируем класс</b>

In [37]:
def dec_speak(cls):
    cls.speak = lambda self, message: "{0} says: {1}".format(
        self.__class__.__name__, message)  #cls.speak это метод класса, и вызываться будет соответственно
    return cls

"""cls.speak можно было бы прописать в классе, напр. так

class NewClass():
    def __init__(self):
        pass

    def speak(self):
        return "{0} says: {1}".format(self.__class__.__name__, message)

"""


Fraction = dec_speak(Fraction)
f1 = Fraction(2, 3)
print(f1.denominator)
print(f1.speak("Hello"))

class Person():
    pass

Person = dec_speak(Person)
p = Person()
p.speak("this is a message")

3
Fraction says: Hello


'Person says: this is a message'

### Debuger

In [40]:
from datetime import datetime, timezone


def info(self):
    result = []
    result.append("time: {}".format(datetime.now(timezone.utc)))
    result.append("Class: {}".format(self.__class__.__name__))
    result.append("id: {}".format(hex(id(self))))
    for k, v in vars(self).items():
        result.append("{}: {}".format(k, v))
    return result


def debug_info(cls):
    cls.debug = info
    return cls


@debug_info
class Person():
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    # этот метод нигде не вызывается, написан просто для наглядности
    def say_hi():
        return "Hallo there!"


p = Person("Max", 1990)
print(p.debug())
# получается, что def info(self) - это метод класса, вынесенный из класса

['time: 2020-05-15 16:08:22.164286+00:00', 'Class: Person', 'id: 0x2478a62ddc8', 'name: Max', 'birth_year: 1990']


In [41]:
from datetime import datetime, timezone


def info(self):
    result = []
    result.append("time: {}".format(datetime.now(timezone.utc)))
    result.append("Class: {}".format(self.__class__.__name__))
    result.append("id: {}".format(hex(id(self))))
    for k, v in vars(self).items():
        result.append("{}: {}".format(k, v))
    return result


def debug_info(cls):
    cls.debug = info
    return cls


@debug_info
class Automobile():
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError("Speed connot exceed top_speed")
        else:
            self._speed = new_speed


favorite = Automobile("Frod", "Model T", 1908, 45)

print(favorite.debug())
favorite.speed = 40
print(favorite.debug())


['time: 2020-05-15 16:47:43.858763+00:00', 'Class: Automobile', 'id: 0x2478a657708', 'make: Frod', 'model: Model T', 'year: 1908', 'top_speed: 45', '_speed: 0']
['time: 2020-05-15 16:47:43.859766+00:00', 'Class: Automobile', 'id: 0x2478a657708', 'make: Frod', 'model: Model T', 'year: 1908', 'top_speed: 45', '_speed: 40']
